# Prediction of Closed Questions on StackOverflow

In [2]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.datasets import make_classification, make_blobs
from matplotlib.colors import ListedColormap
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
import time
import pickle

#### Statuses 

In [ ]:
statuses = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']

####  Function `set_binary` to convert the Multiclass Labels to Binary Labels

In [ ]:
def set_binary(row):
    if row == 3:
        return 0
    else :
        return 1

### Data to use for Training and Validation

1: Balanced Dataset
    
2: Unbalanced Dataset

3: Undersampled Dataset

In [3]:
data_to_use = 2

# Balanced Dataset

It contains 50% Open and 50% Closed .i.e,
89176 closed & 89176 open records

In [ ]:
if(data_to_use == 1 ):
    
    raw_features=pd.read_csv('data/data_small.csv')  #('feature_small_final.csv')
    raw_features=raw_features.drop('Unnamed: 0',axis=1)
    # change comments to switch between binary and non binary labels
    y = raw_features['label']#.apply(set_binary)
    raw_features = raw_features.drop('label',axis=1)
    X = raw_features.as_matrix()

# Imbalanced Dataset

Total Records:

Closed: 89176 

Open  : 

In [4]:
if(data_to_use == 2 ):
    
    raw_features1=pd.read_csv('data/data_large.csv')  #('feature_large_final.csv')
    raw_features1=raw_features1.drop('Unnamed: 0',axis=1)

    y=raw_features1['label']    #.apply(set_binary)   #.values
    raw_features=raw_features1.drop('label',axis=1)
    X=raw_features.as_matrix()

# Undersampling the Large Imbalanced Dataset

In [ ]:
if(data_to_use == 3):
    raw_features1=pd.read_csv('data/data_large.csv')
    raw_features1=raw_features1.drop('Unnamed: 0',axis=1)

In [ ]:
if(data_to_use == 3):

    from collections import Counter
    from imblearn.under_sampling import RandomUnderSampler
    import math

    def plot_pie(y):
        target_stats = Counter(y)
        labels = list(target_stats.keys())
        sizes = list(target_stats.values())
        explode = tuple([0.1] * len(target_stats))

        fig, ax = plt.subplots()
        ax.pie(sizes, explode=explode, labels=labels, shadow=True,
               autopct='%1.1f%%')
        ax.axis('equal')

    Y_ = raw_features1['label']  
    X_ = raw_features1.drop('label',axis=1)

    #ratio = 'auto'
    
    
    # r is the fraction of open records in the required complete data
    f = 1/2
    
    
    m = f/(1-f)
    ratio = {0: math.ceil(Counter(Y_)[1]*m), 1: Counter(Y_)[1]}
    X_res, y_res = RandomUnderSampler(ratio=ratio, random_state=0).fit_sample(X_, Y_)

    y=y_res

    X=X_res

    print('Information of the big data set after balancing using "auto"'
          ' mode:\n ratio={} \n y: {}'.format(ratio, Counter(y_res)))
    plot_pie(y_res)

## Splitting the Data into train and test.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 6)

In [ ]:
print(sum(y_train))
print((y_train.shape))

In [ ]:
# X_train=None
# y_train=None

## Random Forest Classifier

In [6]:
start = time.time()
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1).fit(X_train, y_train)
end = time.time()
print(end - start)

78.19511818885803


In [ ]:
# pickle.dump(clf, open('rf_classifier_lg.pkl', 'wb'))

In [ ]:
# some time later...
 
# load the model from disk
# clf = pickle.load(open('rf_classifier_lg.pkl', 'rb'))

In [7]:
#print('Accuracy of RF classifier on training set: {:.5f}'.format(clf.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.5f}'.format(clf.score(X_test, y_test)))

Accuracy of RF classifier on test set: 0.97530


In [8]:
y_predicted = clf.predict(X_test)

#### Creating Confusion Matrix and Measuring Prescision Recall

In [9]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_predicted)
confusion

array([[    49,      6,      5,   7687,      3],
       [     9,      1,      0,   4139,      0],
       [    11,      3,      3,   4129,      1],
       [   241,     39,     77, 714812,     14],
       [     2,      0,      0,   1740,      1]])

##### Calculating Class Probabilities

In [10]:
y_proba = clf.predict_proba(X_test)

In [11]:
y_proba.shape

(732972, 5)

#### Calculating Loss Function

In [12]:
import sys, csv
from math import log

In [13]:
y_proba[0]

array([ 0.  ,  0.  ,  0.  ,  0.99,  0.01])

In [14]:
# logs = []
# n = 0

# for true_index in y_test.values:    
#     prediction_for_true = y_proba[n][true_index]
#     # print prediction_for_true
#     log_p = log( float( prediction_for_true))
#     logs.append( log_p )
#     n += 1

# logs = sum( logs )
# logloss = - logs / n * 1.0

# print(logloss)
# print()

In [15]:
from sklearn.metrics import log_loss

In [16]:
log_loss(y_test.values,y_proba)

0.42707425649152736